# **ANALISIS SENTIMEN PENGGUNA TERHADAP APLIKASI BRIMO BRI DI PLAY STORE**

## **1. Testing**

## 1.1 Preparation

In [18]:
!pip install Sastrawi
!pip install nltk
import pickle
from pickle import UnpicklingError
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import requests
import json
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Muat model SVM
try:
    with open('svm_model.pkl', 'rb') as f:
        svm = pickle.load(f)
except (EOFError, UnpicklingError) as e:
    print(f"Error loading SVM model: {e}")
    # Additional diagnostic steps:
    print("Check if 'svm_model.pkl' exists and is not empty.")
    print("If the file was transferred, ensure it was done in binary mode.")

# Muat TfidfVectorizer
try:
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        tfidf = pickle.load(f)
except (EOFError, UnpicklingError) as e:
    print(f"Error loading TF-IDF vectorizer: {e}")

# Fungsi pra-pemrosesan teks (sama seperti di Pemodelan.ipynb)
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus mention (@username)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus hashtag (#hashtag)
    text = re.sub(r"http\S+", '', text)  # Menghapus URL
    text = re.sub(r'[0-9]+', '', text)  # Menghapus angka
    text = re.sub(r'[^\w\s]', '', text)  # Menghapus karakter non-alfanumerik kecuali spasi
    text = text.replace('\n', ' ')  # Mengganti newline dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation))  # Menghapus tanda baca
    text = ' '.join([word for word in text.split() if word.lower() not in ["brimo", "bri", "bri mobile"]])  # Menghapus kata-kata yang diinginkan
    text = text.strip(' ')  # Menghapus spasi di awal dan akhir teks
    return text  # Mengembalikan teks yang telah dibersihkan

def casefoldingText(text):
    text = text.lower()  # Mengubah semua karakter dalam teks menjadi huruf kecil
    return text  # Mengembalikan teks yang telah di-casefold

def tokenizingText(text):
    text = word_tokenize(text)  # Memecah teks menjadi token (kata-kata)
    return text  # Mengembalikan daftar token

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))  # Mengambil daftar stopwords bahasa Indonesia
    listStopwords1 = set(stopwords.words('english'))  # Mengambil daftar stopwords bahasa Inggris
    listStopwords.update(listStopwords1)  # Menggabungkan dua daftar stopwords
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku','di','ya','loh','kah','deh'])
    filtered = []
    for txt in text:
        if txt not in listStopwords:  # Memeriksa apakah kata bukan stopword
            filtered.append(txt)  # Menambahkan kata yang bukan stopword ke daftar filtered
    text = filtered  # Menggantikan teks dengan daftar kata yang telah difilter
    return text  # Mengembalikan daftar kata yang telah difilter

def stemmingText(text):
    factory = StemmerFactory()  # Membuat pabrik stemmer
    stemmer = factory.create_stemmer()  # Membuat stemmer
    words = text.split()  # Memecah teks menjadi kata-kata
    stemmed_words = [stemmer.stem(word) for word in words]  # Melakukan stemming pada setiap kata
    stemmed_text = ' '.join(stemmed_words)  # Menggabungkan kata-kata yang telah distem menjadi satu kalimat
    return stemmed_text  # Mengembalikan teks yang telah distem

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)  # Menggabungkan daftar kata menjadi satu kalimat
    return sentence  # Mengembalikan kalimat yang dibentuk dari daftar kata

def fix_slangwords(text):
    words = text.split()  # Memecah teks menjadi kata-kata
    fixed_words = []  # Inisialisasi daftar untuk kata-kata yang sudah diperbaiki
    for word in words:
        if word.lower() in slangwords:  # Memeriksa apakah kata tersebut adalah kata slang
            fixed_words.append(slangwords[word.lower()])  # Jika ya, mengganti kata slang dengan kata standar
        else:
            fixed_words.append(word)  # Jika bukan kata slang atau kata yang ingin dihapus, tetap mempertahankan kata tersebut
    fixed_text = ' '.join(fixed_words)  # Menggabungkan kata-kata yang sudah diperbaiki menjadi satu teks
    return fixed_text  # Mengembalikan teks yang sudah diperbaiki

url = 'https://raw.githubusercontent.com/mashumabduljabbar/dataset/master/slangwords.json'  # URL tempat kamus slangwords disimpan

response = requests.get(url)  # Mengirim permintaan HTTP GET ke URL untuk mendapatkan konten

if response.status_code == 200:  # Jika status code respons adalah 200 (OK)
    try:
        slangwords = json.loads(response.text)  # Mengurai JSON yang diperoleh dari respons dan menyimpannya dalam variabel slangwords
    except json.JSONDecodeError as e: # Menangani kesalahan jika terjadi masalah saat mengurai JSON
        print("Error decoding JSON:", e)
        print("Response content:", response.text)
else:
    print("Failed to fetch data from URL. Status code:", response.status_code) # Menampilkan pesan kesalahan jika tidak berhasil mengambil data dari URL

def preprocess_text(text):
    text = cleaningText(text)
    text = casefoldingText(text)
    text = fix_slangwords(text)
    text = tokenizingText(text)
    text = filteringText(text)
    text = toSentence(text)
    return text

def prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm):
    # Melakukan preprocessing pada kalimat baru
    review_baru_cleaned = cleaningText(review_baru)
    review_baru_casefolded = casefoldingText(review_baru_cleaned)
    review_baru_slangfixed = fix_slangwords(review_baru_casefolded)
    review_baru_tokenized = tokenizingText(review_baru_slangfixed)
    review_baru_filtered = filteringText(review_baru_tokenized)
    review_baru_final = toSentence(review_baru_filtered)

    # Menggunakan objek tfidf yang sudah di-fit dari pelatihan sebelumnya
    X_review_baru = tfidf.transform([review_baru_final])

    # Convert sparse matrix to dense array
    X_review_baru = X_review_baru.toarray()

    # Memperoleh prediksi sentimen review baru menggunakan model terbaik
    prediksi_sentimen = svm.predict(X_review_baru)

    # Menampilkan hasil prediksi
    if prediksi_sentimen[0] == 'positive':
        hasil = "Sentimen review baru adalah POSITIF."
    elif prediksi_sentimen[0] == 'negative':
        hasil = "Sentimen review baru adalah NEGATIF."
    else:
        hasil = "Sentimen review baru adalah NETRAL."

    return hasil

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1.2 Kategori Negatif

In [19]:
review_baru = "susah banget loginnya nih apps gagal mulu"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)

'Sentimen review baru adalah NEGATIF.'

## 1.3 Kategori Positif

In [21]:
review_baru = "Aplikasi yg bermanfaat, membantu transaksi saya tidak pernah gagal"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)

'Sentimen review baru adalah POSITIF.'

## 1.4 Kategori Netral

In [24]:
review_baru = "Sudah oke"
prediksi_sentimen_kalimat_baru(review_baru, tfidf, svm)

'Sentimen review baru adalah NETRAL.'